In [1]:
import os
import csv
import sys
module_path = os.path.abspath(os.path.join('C:\\Users\\koolk\\Desktop\\brain-diffusion\\Chad_functions_and_unittests'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

In [5]:
#The purpose of this book is to compare in one plot the results of varying parameters.

geoM2xy_60nm_100nm_mixed_NPs_in_0_9_glycerol = np.genfromtxt('./60nm_100nm_mixed_NPs_in_0_9_glycerol/geoM2xy_60nm_100nm_mixed__NPs_in_0_9_glycerol.csv',delimiter =",")
geoM2xy_60nm_YG_NPs_in_0_9_glycerol = np.genfromtxt('./60nm_NPs_in_0_9_glycerol/geoM2xy_60nm_YG__NPs_in_0_9_glycerol.csv',delimiter =",")
geoM2xy_100nm_PEG_NPs_in_0_9_glycerol = np.genfromtxt('./100nm_PS_PEG_in_0_9_glycerol/geoM2xy_100nm_PEG_NPs_in_0_9_glycerol.csv',delimiter =",")
geoM2xy_100nm_YG_NPs_in_0_9_glycerol = np.genfromtxt('./100nm_YG_NPs_in_0_9_glycerol/geoM2xy_100nm_YG__NPs_in_0_9_glycerol.csv',delimiter =",")

geoM2xy_100nm_RED_NPs_in_0_9_glycerol = np.genfromtxt('../02-13-17_Glycerol_tests/geoM2xy_confocal_1_0h.csv',delimiter=",")
geoM2xy_100nm_RED_NPs_in_0_75_glycerol = np.genfromtxt('../03_08_16_100nm_PS_NPs_in_glycerol/0_75_glycerol/geoM2xy_100nm_RED__NPs_in_0_75_glycerol.csv',delimiter=",")
# geoM2xy_100nm_PS_PEG_in_midbrain = np.genfromtxt('./100nm_YG_NPs_in_0_9_glycerol/geoM2xy_100nm_PS_PEG_in_midbrain.csv',delimiter =",")

# geoM2xy_100nm_PS_PEG_in_0_4_agarose = np.genfromtxt('../../02-23-17-PS-NPs-and-PS-PEG-NPs-in-0-4-agarose/100nm_PS-PEG/geoM2xy_100nm_PS_PEG_in_0-4_agarose.csv',delimiter =",")
# # geoM2xy_20nm_PS_in_0_4_agarose = np.genfromtxt('./20nm_PS-NPs/geoM2xy_20nm_PS_in_0-4_agarose',delimiter =",")
# geoM2xy_20nm_PS_PEG_in_0_4_agarose = np.genfromtxt('./20nm_PS-PEG/geoM2xy_20nm_PS_PEG_in_0-4_agarose',delimiter =",")

In [3]:
# geoM2xy_camera_1000ms_60x = np.genfromtxt('./geoM2xy_camera_1000ms_60x.csv',delimiter =",")
# arM2xy_camera_1000ms_60x = np.genfromtxt('./arM2xy_camera_1000ms_60x.csv',delimiter =",")

In [7]:
f1 = 3.95 #mixed, YG, 60s
f2 = 2 #PEG
f3 = 1.62
f4 = 2.78

time1 = np.zeros(geoM2xy_60nm_100nm_mixed_NPs_in_0_9_glycerol.shape[0])
time2 = np.zeros(geoM2xy_100nm_PEG_NPs_in_0_9_glycerol.shape[0])
time3 = np.zeros(geoM2xy_100nm_RED_NPs_in_0_9_glycerol.shape[0])
time4 = np.zeros(geoM2xy_100nm_RED_NPs_in_0_75_glycerol.shape[0])
for num in range(0, geoM2xy_60nm_100nm_mixed_NPs_in_0_9_glycerol.shape[0]):
    time1[num] = num/f1

for num in range(0, geoM2xy_100nm_PEG_NPs_in_0_9_glycerol.shape[0]):
    time2[num] = num/f2

for num in range(0, geoM2xy_100nm_RED_NPs_in_0_9_glycerol.shape[0]):
    time3[num] = num/f3

for num in range(0, geoM2xy_100nm_RED_NPs_in_0_75_glycerol.shape[0]):
    time4[num] = num/f4

In [10]:
# Theoretical diffusion calculation using Stokes-Einstein.  Required inputs: TC (Temperature in Celsius), mu1 and mu2
# (viscosities at two temperatures to perform linear regression), TC1 and TC2 (Temperatures at which mu1 and mu2 were
# taken). and radius (radius of particles in m).

#This also has to take into account the percent glycerol in acqueous solution.

TC = 29
perc = 0.9

#95% glycerol and 80% glycerol (Pa*s)

mu1a = 0.236; mu2a = 0.121; TC1 = 30; TC2 = 40; perca = 0.95
mu1b = 0.0338; mu2b = 0.0207; percb = 0.80
mu1c = 0.0042; mu2c = 0.00309; percc = 0.50

mu1 = (perc - percb)*(mu1b - mu1a)/(percb - perca) + mu1b
mu2 = (perc - percb)*(mu2b - mu2a)/(percb - perca) + mu2b

radius = (128/2) * 10**-9 #m
radius2 = (66/2) * 10**-9
radius3 = (138/2) * 10**-9

k = 1.380649 * 10**-23
TK = TC + 273
mu = (mu2 - mu1)*(TC - TC1)/(TC2 - TC1) + mu1

Deff1 = k*TK/(6*np.pi*mu*radius) #m2/s
Deff2 = Deff1 * 10**12 #um2/s

Deff1a = k*TK/(6*np.pi*mu*radius2) #m2/s
Deff2a = Deff1a * 10**12 #um2/s 

Deff1b = k*TK/(6*np.pi*mu*radius3) #m2/s
Deff2b = Deff1b * 10**12 #um2/s 

print('Theoretical diffusion coefficient', np.round(Deff2, 4), 'um2/s')
print(mu1, mu2)

Theoretical diffusion coefficient 0.0196 um2/s
0.16860000000000006 0.08756666666666668


In [11]:
TC = 29
perc = 0.8

#95% glycerol and 80% glycerol (Pa*s)

mu1a = 0.236; mu2a = 0.121; TC1 = 30; TC2 = 40; perca = 0.95
mu1b = 0.0338; mu2b = 0.0207; percb = 0.80
mu1c = 0.0042; mu2c = 0.00309; percc = 0.50

mu1 = (perc - percb)*(mu1b - mu1a)/(percb - perca) + mu1b
mu2 = (perc - percb)*(mu2b - mu2a)/(percb - perca) + mu2b

radius = (128/2) * 10**-9 #m

k = 1.380649 * 10**-23
TK = TC + 273
mu = (mu2 - mu1)*(TC - TC1)/(TC2 - TC1) + mu1

Deff1c = k*TK/(6*np.pi*mu*radius) #m2/s
Deff2c = Deff1c * 10**12 #um2/s

In [12]:
Deff3 = Deff2*np.ones(time1.shape[0])
theo_MSD = 4*np.multiply(Deff3,time1)
# theo_MSD

Deff3a = Deff2a*np.ones(time1.shape[0])
theo_MSDa = 4*np.multiply(Deff3a,time1)

Deff3b = Deff2b*np.ones(time1.shape[0])
theo_MSDb = 4*np.multiply(Deff3b,time1)
# theo_MSDa

Deff3c = Deff2c*np.ones(time4.shape[0])
theo_MSDc = 4*np.multiply(Deff3c,time4)

In [ ]:
#Linear algebra to find Deff (PS-PEG-NPs):
to_frame1 = 30

w = dict()
residuals = dict()
line = dict()
A1 = np.ones((np.shape(time2)[0], 2))
A1[:, 0] = time2

w[0], residuals[0] = np.linalg.lstsq(A1[1:to_frame1, :], geoM2xy_100nm_PEG_NPs_in_0_9_glycerol[1:to_frame1])[0:2]
# w[1], residuals[1] = np.linalg.lstsq(A1[1:to_frame1, :], geoM2xy_100nm_PS_PEG_in_hippocampus[1:to_frame1])[0:2]
# w[2], residuals[2] = np.linalg.lstsq(A1[1:to_frame1, :], geoM2xy_100nm_PS_PEG_in_midbrain[1:to_frame1])[0:2]
# w[1] = np.linalg.lstsq(A1[1:to_frame1, :], geoM2xy_20nm_PS_PEG_in_0_4_agarose[1:to_frame1])[0]

line[0] = w[0][0]*time2 + w[0][1]
# line[1] = w[1][0]*time2 + w[1][1]
# line[2] = w[2][0]*time2 + w[2][1]

In [ ]:
print('Diffusion Coeff (um^2/s):', np.round(w[0][0], 7), 'geoM2xy_100nm_PEG_NPs_in_0_9_glycerol')
# print('Diffusion Coeff (um^2/s):', np.round(w[1][0], 7), 'geoM2xy_100nm_PS_PEG_in_hippocampus')
# print('Diffusion Coeff (um^2/s):', np.round(w[2][0], 7), 'geoM2xy_100nm_PS_PEG_in_midbrain')
# print('Diffusion Coeff (um^2/s):', np.round(w[1][0], 7), 'geoM2xy_20nm_PS_PEG_in_0_4_agarose')

In [ ]:
residuals[0]/17

In [ ]:
#Linear algebra to find Deff (PS NPs):
to_frame = 30

w1 = dict()
residuals1 = dict()
line1 = dict()
A2 = np.ones((np.shape(time1)[0], 2))
A2[:, 0] = time1


w1[0], residuals1[0] = np.linalg.lstsq(A2[1:to_frame, :], geoM2xy_60nm_100nm_mixed_NPs_in_0_9_glycerol[1:to_frame])[0:2]
w1[1], residuals1[1] = np.linalg.lstsq(A2[1:to_frame, :], geoM2xy_60nm_YG_NPs_in_0_9_glycerol[1:to_frame])[0:2]
w1[2], residuals1[2] = np.linalg.lstsq(A2[1:to_frame, :], geoM2xy_100nm_YG_NPs_in_0_9_glycerol[1:to_frame])[0:2]


line1[0] = w1[0][0]*time1 + w1[0][1]
line1[1] = w1[1][0]*time1 + w1[1][1]
line1[2] = w1[2][0]*time1 + w1[2][1]

In [ ]:
print('Diffusion Coeff (um^2/s):', np.round(w1[0][0], 7), 'geoM2xy_60nm_100nm_mixed_NPs_in_0_9_glycerol')
print('Diffusion Coeff (um^2/s):', np.round(w1[1][0], 7), 'geoM2xy_60nm_YG_NPs_in_0_9_glycerol')
print('Diffusion Coeff (um^2/s):', np.round(w1[2][0], 7), 'geoM2xy_100nm_YG_NPs_in_0_9_glycerol')

In [ ]:
residuals1[0]/20

In [ ]:
# N = 5

# ind = np.arange(N)
# width = 0.35     

# x1 = [1, 2, 3, 4]
# x2 = [1, 2, 3, 4]
# y1 = [w[0][0], w[1][0], w1[0][0], w1[1][0]]
# y2 = [w[2][0], w[3][0], w1[2][0], w1[3][0]]

# labels1 = ['FM2xy_camera_1000ms_40x', 'FM2xy_camera_1000ms_60x', 'FM2xy_camera_625ms_40x', 'FM2xy_camera_625ms_60x']
# labels2 = ['FM2xy_confocal_1000ms_40x', 'FM2xy_confocal_1000ms_60x', 'FM2xy_confocal_625ms_40x', 'FM2xy_confocal_625ms_60x']

In [ ]:
# fig = plt.figure()
# ax = fig.add_subplot(111)

# ## the data
# N = 4

# ## necessary variables
# ind = np.arange(N)                # the x locations for the groups
# width = 0.6                     # the width of the bars

# ## the bars
# rects1 = ax.bar(ind, y1, width,
#                 color='green')

# # axes and labels
# ax.set_xlim(-width/2,len(ind)+width/2)
# ax.set_ylim(0,0.0012)
# ax.set_ylabel('Diffusion Coefficient (um^2/s)')
# ax.set_title('Diffusion coefficients for camera')
# xTickMarks = [labels1[i-1] for i in range(1,5)]
# ax.set_xticks(ind+width)
# xtickNames = ax.set_xticklabels(xTickMarks)
# plt.setp(xtickNames, rotation=45, fontsize=10)

# plt.show()

In [ ]:
# fig = plt.figure()
# ax = fig.add_subplot(111)

# ## the data
# N = 4

# ## necessary variables
# ind = np.arange(N)                # the x locations for the groups
# width = 0.6                     # the width of the bars

# ## the bars
# rects1 = ax.bar(ind, y2, width,
#                 color='green')

# # axes and labels
# ax.set_xlim(-width/2,len(ind)+width/2)
# ax.set_ylim(0,0.0012)
# ax.set_ylabel('Diffusion Coefficient (um^2/s)')
# ax.set_title('Diffusion coefficients for confocal')
# xTickMarks = [labels2[i-1] for i in range(1,5)]
# ax.set_xticks(ind+width)
# xtickNames = ax.set_xticklabels(xTickMarks)
# plt.setp(xtickNames, rotation=45, fontsize=10)

# plt.show()

In [32]:
#Figure parameters

filename = 'geo_comparison_60_100_mixed.png'

limit1=10.1 #x range
tick1 = round(limit1)/5 #x ticks

limit2=1.6 #y range
tick2 = round(limit2/5, 0) + 0.2 #y ticks

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=1 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
def color_variant(hex_color, brightness_offset=1):
    """ takes a color like #87c95f and produces a lighter or darker variant """
    if len(hex_color) != 7:
        raise Exception("Passed %s into color_variant(), needs to be in #87c95f format." % hex_color)
    rgb_hex = [hex_color[x:x+2] for x in [1, 3, 5]]
    new_rgb_int = [int(hex_value, 16) + brightness_offset for hex_value in rgb_hex]
    new_rgb_int = [min([255, max([0, i])]) for i in new_rgb_int] # make sure new values are between 0 and 255
    # hex() produces "0x88", we want just "88"
    return "#" + "".join([hex(i)[2:] for i in new_rgb_int])

In [ ]:
# for name, hex in mpl.colors.cnames.items():
#     print(name, hex)

# g2 = color_variant('#008000', brightness_offset=0)
# b2 = color_variant('#0000FF', brightness_offset=0)

In [ ]:
def clamp(val, minimum=0, maximum=255):
    if val < minimum:
        return minimum
    if val > maximum:
        return maximum
    return val

def colorscale(hexstr, scalefactor):
    """
    Scales a hex string by ``scalefactor``. Returns scaled hex string.

    To darken the color, use a float value between 0 and 1.
    To brighten the color, use a float value greater than 1.

    >>> colorscale("#DF3C3C", .5)
    #6F1E1E
    >>> colorscale("#52D24F", 1.6)
    #83FF7E
    >>> colorscale("#4F75D2", 1)
    #4F75D2
    """

    hexstr = hexstr.strip('#')

    if scalefactor < 0 or len(hexstr) != 6:
        return hexstr

    r, g, b = int(hexstr[:2], 16), int(hexstr[2:4], 16), int(hexstr[4:], 16)

    r = round(clamp(r * scalefactor))
    g = round(clamp(g * scalefactor))
    b = round(clamp(b * scalefactor))

    return "#%02x%02x%02x" % (r, g, b)

In [ ]:
g2 = colorscale('#008000', 1)
b2 = colorscale('#0000FF', 10)
r2 = colorscale('#FF0000', 10)
g2

In [33]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time1, geoM2xy_20nm_PS_in_0_4_agarose, 'bo', ms=mi, label='20nm PS NPs')
# ax.plot(time2, geoM2xy_20nm_PS_PEG_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='20 nm PS-PEG NPs')
# ax.plot(time1, geoM2xy_100nm_PS_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='100nm PS NPs')
ax.plot(time1[0:], geoM2xy_60nm_YG_NPs_in_0_9_glycerol[0:], 'g--', linewidth=wi, dashes=[d1,d2], label='60nm YG NPs')
ax.plot(time1[0:], geoM2xy_60nm_100nm_mixed_NPs_in_0_9_glycerol[0:], 'b--', linewidth=wi, dashes=[d1,d2], label='60/100nm YG mix')
ax.plot(time1[0:], geoM2xy_100nm_YG_NPs_in_0_9_glycerol[0:], 'r--', linewidth=wi, dashes=[d1,d2], label='100nm YG NPs')

# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)

ax.plot(time1[1:], theo_MSD[1:], '-', linewidth=10, label='theoretical 100nm', color='red')
ax.plot(time1[1:], theo_MSDa[1:], '-', linewidth=10, label='theoretical 60nm', color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.65), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
#Figure parameters

filename = 'geo_comparison_YG_Red.png'

limit1=20.1 #x range
tick1 = 4

limit2=2.1 #y range
tick2 =0.5

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=1 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [ ]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time1, geoM2xy_20nm_PS_in_0_4_agarose, 'bo', ms=mi, label='20nm PS NPs')
# ax.plot(time2, geoM2xy_20nm_PS_PEG_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='20 nm PS-PEG NPs')
# ax.plot(time1, geoM2xy_100nm_PS_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='100nm PS NPs')
ax.plot(time3[0:], geoM2xy_100nm_RED_NPs_in_0_9_glycerol[0:], 'b--', linewidth=wi, dashes=[d1,d2], label='100nm Red')
ax.plot(time1[0:], geoM2xy_100nm_YG_NPs_in_0_9_glycerol[0:], 'g--', linewidth=wi, dashes=[d1,d2], label='100nm YG')

# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)

ax.plot(time1[1:], theo_MSD[1:], '-', linewidth=10, label='theoretical 100nm', color='purple')
# ax.plot(time1[1:], theo_MSDa[1:], '-', linewidth=10, label='theoretical 60nm', color='purple')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.65), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [30]:
#Figure parameters

filename = 'geo_comparison_PEG.png'

limit1=20.1 #x range
tick1 = 4

limit2=2.1 #y range
tick2 =0.5

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=1 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [31]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time1, geoM2xy_20nm_PS_in_0_4_agarose, 'bo', ms=mi, label='20nm PS NPs')
# ax.plot(time2, geoM2xy_20nm_PS_PEG_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='20 nm PS-PEG NPs')
# ax.plot(time1, geoM2xy_100nm_PS_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='100nm PS NPs')
ax.plot(time3[0:], geoM2xy_100nm_RED_NPs_in_0_9_glycerol[0:], 'b--', linewidth=wi, dashes=[d1,d2], label='100nm PS')
ax.plot(time2[0:], geoM2xy_100nm_PEG_NPs_in_0_9_glycerol[0:], 'g--', linewidth=wi, dashes=[d1,d2], label='100nm PS-PEG')

# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)

ax.plot(time1[1:], theo_MSD[1:], '-', linewidth=10, label='theoretical 100nm PS', color='blue')
ax.plot(time1[1:], theo_MSDb[1:], '-', linewidth=10, label='theoretical 100nm PS-PEG', color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.75), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [25]:
#Figure parameters

filename = 'geo_comparison_viscous.png'

limit1=10.1 #x range
tick1 = 2

limit2=4.1 #y range
tick2 =1

label=95 #label font size
leg=40 #legend font size
ylabel=r'MSD ($\mu$m$^2$)'

dec1=0 #decimals displayed on x axis
dec2=1 #decimals displayed on y axis

d1 = 30 #dashed line dash length
d2 = 15 #dashed line space length

wi = 10 #line width
mi = 12 #marker size

In [29]:
# Creates figure
fig = plt.figure(figsize=(18, 18), dpi=80)
ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time1, geoM2xy_20nm_PS_in_0_4_agarose, 'bo', ms=mi, label='20nm PS NPs')
# ax.plot(time2, geoM2xy_20nm_PS_PEG_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='20 nm PS-PEG NPs')
# ax.plot(time1, geoM2xy_100nm_PS_in_0_4_agarose, 'b--', linewidth=wi, dashes=[d1,d2], label='100nm PS NPs')
ax.plot(time3[0:], geoM2xy_100nm_RED_NPs_in_0_9_glycerol[0:], 'b--', linewidth=wi, dashes=[d1,d2], label='in 90% glycerol')
ax.plot(time4[0:], geoM2xy_100nm_RED_NPs_in_0_75_glycerol[0:], 'g--', linewidth=wi, dashes=[d1,d2], label='in 75% glycerol')

# ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time1[1:], line1[0][1:], linewidth=wi-3, color=b2)
# ax.plot(time1[1:], line1[1][1:], linewidth=wi-3, color=g2)
# ax.plot(time1[1:], line1[2][1:], linewidth=wi-3, color=r2)

ax.plot(time1[1:], theo_MSD[1:], '-', linewidth=10, label='theoretical in 90% glycerol', color='blue')
ax.plot(time4[1:], theo_MSDc[1:], '-', linewidth=10, label='theoretical in 75% glycerol', color='green')
    
# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(0, limit1, tick1)
ymajor_ticks = np.arange(0, limit2, tick2)

ax.set_xticks(xmajor_ticks)
ax.set_yticks(ymajor_ticks)
ax.title.set_fontsize(70)
ax.set_xlabel('Time (s)', fontsize=label)
ax.set_ylabel(ylabel, fontsize=label)
ax.tick_params(direction='out', pad=16)
ax.legend(loc=(0.02, 0.75), prop={'size': leg})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([0, limit1])
plt.gca().set_ylim([0.00001, limit2])

# Save your figure
plt.savefig('{}'.format(filename), bbox_inches='tight')
plt.show()

In [ ]:
geoM2xy_100nm_PS_PEG_in_0_4_agarose.shape

In [ ]:
geoM2xy_100nm_PS_PEG_in_0_4_agarose

In [ ]:
# #Figure parameters

# filename = 'FBF_comparison_60x.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [ ]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2, FM2xy_camera_1000ms_60x, 'bo', ms=mi, label='Camera 1000ms 60x')
# ax.plot(time1, FM2xy_camera_625ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
# ax.plot(time2, FM2xy_confocal_1000ms_60x, 'go', ms=mi, label='Confocal 1000ms 60x')
# ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [ ]:
# #Figure parameters

# filename = 'FBF_comparison_625ms.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [ ]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time1, FM2xy_camera_625ms_40x, 'bo', ms=mi, label='Camera 625ms 40x')
# ax.plot(time1, FM2xy_camera_625ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
# ax.plot(time1, FM2xy_confocal_625ms_40x[:-1], 'go', ms=mi, label='Confocal 625ms 40x')
# ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [ ]:
# #Figure parameters

# filename = 'FBF_comparison_1000ms.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [ ]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2, FM2xy_camera_1000ms_40x, 'bo', ms=mi, label='Camera 1000ms 40x')
# ax.plot(time2, FM2xy_camera_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 1000ms 60x')
# ax.plot(time2, FM2xy_confocal_1000ms_40x, 'go', ms=mi, label='Confocal 1000ms 40x')
# ax.plot(time2, FM2xy_confocal_1000ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 1000ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [ ]:
# #Figure parameters

# filename = 'FBF_comparison_camera.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [ ]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2, FM2xy_camera_1000ms_40x, 'bo', ms=mi, label='Camera 1000ms 40x')
# ax.plot(time2, FM2xy_camera_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Camera 1000ms 60x')
# ax.plot(time1, FM2xy_camera_625ms_40x, 'go', ms=mi, label='Camera 625ms 40x')
# ax.plot(time1, FM2xy_camera_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Camera 625ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [ ]:
# #Figure parameters

# filename = 'FBF_comparison_confocal.png'

# limit1=50.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.051 #y range
# tick2 = round(limit2, 2)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=2 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [ ]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2, FM2xy_confocal_1000ms_40x, 'bo', ms=mi, label='Confocal 1000ms 40x')
# ax.plot(time2, FM2xy_confocal_1000ms_60x, 'b--', linewidth=wi, dashes=[d1,d2], label='Confocal 1000ms 60x')
# ax.plot(time1, FM2xy_confocal_625ms_40x[:-1], 'go', ms=mi, label='Confocal 625ms 40x')
# ax.plot(time1, FM2xy_confocal_625ms_60x, 'g--', linewidth=wi, dashes=[d1,d2], label='Confocal 625ms 60x')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# ax.plot(time2, line[2], linewidth=3, color='purple')
# ax.plot(time2, line[3], linewidth=3, color='purple')
# ax.plot(time1, line1[2], linewidth=3, color='purple')
# ax.plot(time1, line1[3], linewidth=3, color='purple')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()

In [ ]:
# #Figure parameters

# filename = 'FBF_geo_ar_comparison.png'

# limit1=20.1 #x range
# tick1 = round(limit1)/5 #x ticks

# limit2=0.0051 #y range
# tick2 = round(limit2, 3)/5 #y ticks

# label=95 #label font size
# leg=40 #legend font size
# ylabel=r'MSD ($\mu$m$^2$)'

# dec1=0 #decimals displayed on x axis
# dec2=3 #decimals displayed on y axis

# d1 = 30 #dashed line dash length
# d2 = 15 #dashed line space length

# wi = 10 #line width
# mi = 12 #marker size

In [ ]:
# # Creates figure
# fig = plt.figure(figsize=(24, 18), dpi=80)
# ax = fig.add_subplot(111)
#     # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# ax.plot(time2[0:19], geoM2xy_camera_1000ms_60x[0:-1], 'b--', linewidth=wi, dashes=[d1,d2], label='Geometric')
# ax.plot(time2[0:19], arM2xy_camera_1000ms_60x[0:-1], 'g--', linewidth=wi, dashes=[d1,d2], label='Arithmetic')
# ax.plot(time2, FM2xy_camera_1000ms_60x, 'r--', linewidth=wi, dashes=[d1,d2], label='Frame-by-frame')
    
# # ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
# # ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
# # ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')

# # ax.plot(t, line[0], linewidth=3, color='blue')
# # ax.plot(t, line[1], linewidth=3, color='red')
# # ax.plot(t, line[2], linewidth=3, color='green')
    
# #ax.plot(t, linel[0], linewidth=3, color='blue')
# #ax.plot(t, linel[1], linewidth=3, color='red')
# #ax.plot(t, linel[2], linewidth=3, color='green')
    
# # A few adjustments to prettify the graph
# for item in ([ax.xaxis.label, ax.yaxis.label] +
#              ax.get_xticklabels() + ax.get_yticklabels()):
#     item.set_fontsize(70)

# xmajor_ticks = np.arange(0, limit1, tick1)
# ymajor_ticks = np.arange(0, limit2, tick2)

# ax.set_xticks(xmajor_ticks)
# ax.set_yticks(ymajor_ticks)
# ax.title.set_fontsize(70)
# ax.set_xlabel('Time (s)', fontsize=label)
# ax.set_ylabel(ylabel, fontsize=label)
# ax.tick_params(direction='out', pad=16)
# ax.legend(loc=(0.06, 0.66), prop={'size': leg})
# plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))


# #plt.yscale('log')
# #plt.xscale('log')
# plt.gca().set_xlim([0, limit1])
# plt.gca().set_ylim([0, limit2])

# # Save your figure
# plt.savefig('{}.png'.format(filename), bbox_inches='tight')
# plt.show()